In [4]:
from basketball_reference_web_scraper import client

from basketball_reference_web_scraper.data import Location
from basketball_reference_web_scraper.data import Team
from basketball_reference_web_scraper.data import OutputType
from basketball_reference_web_scraper.data import Position

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import matplotlib.pyplot as plt

import pandas as pd

from matplotlib import pyplot as plt

%matplotlib inline

In [102]:
client.players_advanced_season_totals(season_end_year=2001, output_type=OutputType.CSV, output_file_path="./2000-2001_advanced_player_season_totals.csv")
client.players_season_totals(season_end_year=2010, output_type=OutputType.CSV, output_file_path="./2000-2001_player_season_totals.csv" )

In [103]:
data = pd.read_csv("2000-2001_advanced_player_season_totals.csv")
print(data.shape)
data2 = pd.read_csv("2000-2001_player_season_totals.csv")
print(data2.shape)

data["made_three_point_field_goals"] = data2["made_three_point_field_goals"]
data["attempted_three_point_field_goals"] = data2["attempted_three_point_field_goals"]
data["games_started"] = data2["games_started"]
data["attempted_field_goals"] = data2["attempted_field_goals"]
data["made_field_goals"] = data2["made_field_goals"]

print(data.shape)
data.head()

# "made_three_point_field_goals", "attempted_three_point_field_goals", "games_started", "attempted_field_goals","made_field_goals"


(490, 28)
(512, 22)
(490, 33)


,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,...,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals,made_three_point_field_goals,attempted_three_point_field_goals,games_started,attempted_field_goals,made_field_goals
0,abdulma02,Mahmoud Abdul-Rauf,POINT GUARD,31,VANCOUVER GRIZZLIES,41,486,16.7,0.514,0.057,...,1.1,-2.1,-1.0,0.1,False,108,249,75,585,272
1,abdulta01,Tariq Abdul-Wahad,SHOOTING GUARD,26,DENVER NUGGETS,29,420,5.8,0.438,0.090,...,-5.6,-0.3,-5.8,-0.4,False,0,0,0,10,5
2,abdursh01,Shareef Abdur-Rahim,SMALL FORWARD,24,VANCOUVER GRIZZLIES,81,3241,19.1,0.549,0.050,...,1.7,0.1,1.8,3.1,False,5,16,78,1169,579
3,alexaco01,Cory Alexander,POINT GUARD,27,ORLANDO MAGIC,26,227,6.3,0.407,0.286,...,-7.2,1.2,-6.0,-0.2,False,0,1,0,6,1
4,alexaco02,Courtney Alexander,POINT GUARD,23,DALLAS MAVERICKS,38,472,6.8,0.404,0.056,...,-5.5,-0.8,-6.2,-0.5,False,1,6,3,116,46


In [104]:
#Data cleaning

position_dict = {"POINT GUARD": 0, "SHOOTING GUARD": 1, "SMALL FORWARD": 2, "POWER FORWARD": 3, "CENTER": 4, "FORWARD": 5, "GUARD": 6} 

data = data[data['games_played'] >= 30]
data = data.replace(position_dict)
data = data[data['positions'] <= 4]
print(data.shape)
print(data["positions"].max())
data.head()

(345, 33)
4


,slug,name,positions,age,team,games_played,minutes_played,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,...,offensive_box_plus_minus,defensive_box_plus_minus,box_plus_minus,value_over_replacement_player,is_combined_totals,made_three_point_field_goals,attempted_three_point_field_goals,games_started,attempted_field_goals,made_field_goals
0,abdulma02,Mahmoud Abdul-Rauf,0,31,VANCOUVER GRIZZLIES,41,486,16.7,0.514,0.057,...,1.1,-2.1,-1.0,0.1,False,108,249,75,585,272
2,abdursh01,Shareef Abdur-Rahim,2,24,VANCOUVER GRIZZLIES,81,3241,19.1,0.549,0.050,...,1.7,0.1,1.8,3.1,False,5,16,78,1169,579
4,alexaco02,Courtney Alexander,0,23,DALLAS MAVERICKS,38,472,6.8,0.404,0.056,...,-5.5,-0.8,-6.2,-0.5,False,1,6,3,116,46
6,allenra02,Ray Allen,1,25,MILWAUKEE BUCKS,82,3129,22.9,0.610,0.357,...,5.7,-0.3,5.5,5.9,False,0,5,8,253,129
7,alstora01,Rafer Alston,0,24,MILWAUKEE BUCKS,37,288,12.1,0.429,0.357,...,-1.6,-0.5,-2.1,0.0,False,28,87,13,277,95


In [105]:
data = data[['player_efficiency_rating', 'true_shooting_percentage', 'three_point_attempt_rate', 'free_throw_attempt_rate', 'offensive_rebound_percentage', 'defensive_rebound_percentage', 'total_rebound_percentage', 'assist_percentage', 'steal_percentage', 'block_percentage', 'turnover_percentage', 'usage_percentage', 'offensive_win_shares', 'defensive_win_shares', 'win_shares', 'win_shares_per_48_minutes', 'offensive_box_plus_minus', 'defensive_box_plus_minus', 'box_plus_minus', 'value_over_replacement_player', 'positions']]
data.head()
data.to_csv("./2000-2001_min30_advanced_player_season_totals.csv", index=False)